In [1]:
import numpy as np
import scipy.sparse as sparse
import scipy.stats as stats

In [122]:
def svd_rank_one_update(U, S, V, a):
    """
    Computes SVD of USV^T + a1^T via rank one update.
    """
    p, r = U.shape
    q, _ = V.shape

    # Make sure this constitutes a valid SVD
    assert S.shape == (r,)
    assert a.shape == (p,)
    assert U.shape == (p, r)
    assert V.shape == (q, r)

    # Orthogonal projection vectors
    m = U.T @ a
    P = a - U @ m
    R_a = np.linalg.norm(P)
    P /= R_a

    n = V.sum(axis=0)
    Q = 1 - V @ n
    R_b = np.linalg.norm(Q)
    Q /= R_b


    # Create the K that should be eigendecomped
    K1 = np.zeros((r+1, r+1))
    np.fill_diagonal(K1[:r, :r], S)
    K2 = (
        np.concatenate((m, np.array([R_a]))).reshape(-1, 1)
        @ np.concatenate((n, np.array([R_b]))).reshape(1, -1)
    )
    K = K1 + K2

    # Inner eigendecomp
    Up, Sf, VpT = np.linalg.svd(K)
    Vp = VpT.T

    # Results
    Uf = np.hstack((U, P.reshape(-1, 1))) @ Up
    Vf = np.hstack((V, Q.reshape(-1, 1))) @ Vp
    
    return Uf, Sf, Vf

In [187]:
p, q, r = 100, 600, 40
s = 0.1

X = np.random.random((p, q))
X[np.random.random((p, q)) < s] = 0
X = sparse.csr_array(X)

X_ = stats.rankdata(X.toarray(), axis=1)
X_ = stats.norm.ppf(X_ / (q+1))

# Get what zero was mapped to in each row
# (This counts the number of elements less than zero, and adds one
# so that ranking starts at 1)
rank = (X_ < 0).sum(axis=1) + 1
zeromaps = stats.norm.ppf(rank / (q + 1))

X = X_ - zeromaps.reshape(-1, 1)



U, S, VT = np.linalg.svd(X, full_matrices=False)
U = U[:, :r]
S = S[:r]
V = VT[:r, :].T

# Only compare to low-rank matrix:
X = U @ np.diag(S) @ V.T

# Rank one update
a = np.random.random(p)
Uf, Sf, Vf = svd_rank_one_update(U, S, V, a)
Xf = Uf @ np.diag(Sf) @ Vf.T

# Ground truth
Xt = X + a.reshape(-1, 1)
Ut, St, VtT = np.linalg.svd(Xt)
St = St[:r+1]
Ut = Ut[:, :r+1]
Vt = VtT.T[:, :r+1]

# Results
Sdiff = np.linalg.norm(St.flatten() - Sf.flatten())
Udiff = np.linalg.norm(abs(Ut.T@Uf) - np.eye(r+1))
Vdiff = np.linalg.norm(abs(Vt.T@Vf) - np.eye(r+1))
print(Sdiff)
print(Udiff)
print(Vdiff)

1.7613722400597766e-13
1.900233355527885e-12
1.9064278741669653e-12


In [120]:
# Online Python compiler (interpreter) to run Python online.
# Write Python 3 code in this online editor and run it.
# Get started with interactive Python!
# Supports Python Modules: builtins, math,pandas, scipy 
# matplotlib.pyplot, numpy, operator, processing, pygal, random, 
# re, string, time, turtle, urllib.request
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp

p = 50
q = 100
r = 5
s = 0.1 # sparsity percentage

X = np.random.random((p, q))
X[np.random.random((p, q)) < s] = 0
#print(sp.sparse.coo_array(X))
X_ = sp.stats.rankdata(X, axis=1)
X_ = sp.stats.norm.ppf(X_ / (q+1))

# Get what zero was mapped to in each row
zeromaps = np.zeros(p)
for colX, colX_ in zip(X.T, X_.T):
  for row, (iX, iX_) in enumerate(zip(colX, colX_)):
    if iX == 0 and zeromaps[row] == 0:
      zeromaps[row] = iX_

X = X_ - zeromaps.reshape(-1, 1)

# # Create random matrix
# U = sp.stats.ortho_group.rvs(p)[:, :r]
# V = sp.stats.ortho_group.rvs(q)[:, :r]
# S = np.diag(np.random.random(r))
# X = U @ S @ V.T

U, S, VT = np.linalg.svd(X)
V = VT.T
U = U[:, :r]
V = V[:, :r]
S = np.diag(S[:r])

# Only want to compare against low-rank approximation
X = U @ S @ V.T


# Sum terms
a = zeromaps

OLD = False
if OLD:
  # Orthogonal projection vectors
  m = U.T @ a
  P = a - U @ m
  R_a = np.linalg.norm(P)
  P /= R_a

  n = V.sum(axis=0)
  Q = 1 - V @ n
  R_b = np.linalg.norm(Q)
  Q /= R_b


  # Create the K that should be eigendecomped
  K1 = np.zeros((r+1, r+1))
  K1[:r, :r] = S
  K2 = (
    np.concatenate((m, np.array([R_a]))).reshape(-1, 1)
    @ np.concatenate((n, np.array([R_b]))).reshape(1, -1)
  )
  K = K1 + K2

  # Inner eigendecomp
  Up, Sp, VpT = np.linalg.svd(K)
  Vp = VpT.T

  # Results
  Sf = np.diag(Sp)
  Uf = np.hstack((U, P.reshape(-1, 1))) @ Up
  Vf = np.hstack((V, Q.reshape(-1, 1))) @ Vp
else:
  Uf, Sf, Vf = svd_rank_one_update(U, np.diag(S), V, a)
  #Sf = np.diag(Sf)

# Ground truth
Xt = X + a.reshape(-1, 1)
Ut, St, VtT = np.linalg.svd(Xt)
St = np.diag(St[:r+1])
Ut = Ut[:, :r+1]
Vt = VtT.T[:, :r+1]


# Performance
Sdiff = np.linalg.norm(St.flatten() - Sf.flatten())
Udiff = np.linalg.norm(abs(Uf.T@Ut) - np.eye(r+1))
Vdiff = np.linalg.norm(abs(Vf.T@Vt) - np.eye(r+1))
print(Sdiff)
print(Udiff)
print(Vdiff)

#print(Xt - Uf @ Sf @ Vf.T)



2.9545285502658894e-14
2.835927519911171e-13
6.968312007782907e-14


In [157]:
X = np.random.random((p, q))
X[np.random.random((p, q)) < s] = 0
X = sp.sparse.coo_array(X)
print(abs(X).argmin(axis=1))

[[ 0]
 [16]
 [ 9]
 [11]
 [ 5]
 [ 8]
 [24]
 [ 8]
 [ 3]
 [ 2]
 [ 7]
 [ 3]
 [ 6]
 [ 4]
 [ 7]
 [ 1]
 [ 9]
 [ 6]
 [ 5]
 [ 2]
 [ 0]
 [ 0]
 [64]
 [ 7]
 [21]
 [ 0]
 [ 7]
 [32]
 [ 4]
 [ 1]
 [ 3]
 [16]
 [16]
 [21]
 [16]
 [11]
 [ 3]
 [13]
 [ 6]
 [15]
 [23]
 [ 3]
 [ 4]
 [ 4]
 [ 2]
 [ 1]
 [ 0]
 [ 0]
 [ 1]
 [23]
 [ 3]
 [ 0]
 [ 5]
 [ 4]
 [22]
 [18]
 [ 2]
 [ 6]
 [ 0]
 [33]
 [ 1]
 [ 1]
 [23]
 [14]
 [ 1]
 [ 9]
 [ 8]
 [ 2]
 [ 2]
 [11]
 [ 1]
 [ 9]
 [15]
 [ 0]
 [12]
 [24]
 [ 6]
 [10]
 [ 3]
 [ 1]
 [ 5]
 [ 1]
 [28]
 [ 2]
 [ 2]
 [ 0]
 [ 7]
 [15]
 [20]
 [ 6]
 [ 5]
 [24]
 [ 4]
 [ 1]
 [ 2]
 [ 2]
 [ 1]
 [15]
 [ 3]
 [30]]
